In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


In [8]:
import sys
import os
import pandas as pd
import joblib

from data_processing.load_data import load_data
from data_processing.clean_data import clean_data_runs, clean_data_races, clean_before_fitting
from data_processing.merge_data import merge_data
from data_processing.preprocess_data import preprocess_data, split_train_test
from ml_models.linear_regression import train_linear_regression
from metrics.performance_metrics import prepare_predictions, compute_top_prediction, finalize_dataframe, compute_accuracy

In [9]:
# Aller au dossier racine du projet
project_root = os.path.abspath("..")  # Remonte au dossier principal du projet
src_path = os.path.join(project_root, "src")

# Ajouter le chemin src/ au path Python (évite les doublons)
if src_path not in sys.path:
    sys.path.append(src_path)

# Prédictions
model_path = os.path.abspath(os.path.join("..", "models", "linear_regression.pkl"))
model = joblib.load(model_path)

In [ ]:



# Charger les datasets
print("Chargement des données...")
df_runs = load_data("../data/raw/runs.csv")
df_races = load_data("../data/raw/races.csv")

# Nettoyage des données
print("Nettoyage des données...")
clean_data_runs(df_runs)
clean_data_races(df_races)

df = merge_data(df_runs, df_races)
clean_before_fitting(df)

# Prétraitement des données
print("Prétraitement des données...")
df = preprocess_data(df)

# Split en train/test
print("Séparation des données...")
y_train, y_test, X_train, X_test, df_train, df_test = split_train_test(df, train_ratio=0.8)

df_test_original = df_test.copy()

# Entraînement du modèle
print("Entraînement du modèle de régression linéaire...")
train_linear_regression(X_train, y_train)

# Faire des prédictions sur X_test
y_predict = model.predict(X_test)

# Prédiction et évaluation
df = prepare_predictions(df_test_original, y_predict)
df = compute_top_prediction(df)
df_stats = finalize_dataframe(df)

accuracy = compute_accuracy(df_stats)

print(f"✅ Précision basée sur les courses : {accuracy:.2f}%")

Chargement des données...
Nettoyage des données...
Prétraitement des données...
Séparation des données...
Entraînement du modèle de régression linéaire...
Successfully trained linear_regression model
✅ Précision basée sur les courses : 23.88%
